In [11]:
%pip install faiss-gpu chromadb==0.3.21
cache_dir='cache_dir'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
from datasets import load_dataset

In [13]:
import pandas as pd

pdf = pd.read_csv(f"data/labelled_newscatcher_dataset.csv", sep=";")
pdf['id'] = pdf.index
display(pdf)

,topic,link,domain,published_date,title,lang,id
0,SCIENCE,https://www.eurekalert.org/pub_releases/2020-0...,eurekalert.org,2020-08-06 13:59:45,A closer look at water-splitting's solar fuel ...,en,0
1,SCIENCE,https://www.pulse.ng/news/world/an-irresistibl...,pulse.ng,2020-08-12 15:14:19,"An irresistible scent makes locusts swarm, stu...",en,1
2,SCIENCE,https://www.express.co.uk/news/science/1322607...,express.co.uk,2020-08-13 21:01:00,Artificial intelligence warning: AI will know ...,en,2
3,SCIENCE,https://www.ndtv.com/world-news/glaciers-could...,ndtv.com,2020-08-03 22:18:26,Glaciers Could Have Sculpted Mars Valleys: Study,en,3
4,SCIENCE,https://www.thesun.ie/tech/5742187/perseid-met...,thesun.ie,2020-08-12 19:54:36,Perseid meteor shower 2020: What time and how ...,en,4
...,...,...,...,...,...,...,...
108769,NATION,https://www.vanguardngr.com/2020/08/pdp-govern...,vanguardngr.com,2020-08-08 02:40:00,PDP governors’ forum urges security agencies t...,en,108769
108770,BUSINESS,https://www.patentlyapple.com/patently-apple/2...,patentlyapple.com,2020-08-08 01:27:12,"In Q2-20, Apple Dominated the Premium Smartpho...",en,108770
108771,HEALTH,https://www.belfastlive.co.uk/news/health/coro...,belfastlive.co.uk,2020-08-12 17:01:00,Coronavirus Northern Ireland: Full breakdown s...,en,108771
108772,ENTERTAINMENT,https://www.thenews.com.pk/latest/696364-paul-...,thenews.com.pk,2020-08-05 04:59:00,Paul McCartney details post-Beatles distress a...,en,108772


### Vector Library: FAISS

Vector libraries are often sufficient for small, static data. Since it's not a full-fledged database solution, it doesn't have the CRUD (Create, Read, Update, Delete) support. Once the index has been built, if there are more vectors that need to be added/removed/edited, the index has to be rebuilt from scratch. 

That said, vector libraries are easy, lightweight, and fast to use. Examples of vector libraries are [FAISS](https://faiss.ai/), [ScaNN](https://github.com/google-research/google-research/tree/master/scann), [ANNOY](https://github.com/spotify/annoy), and [HNSM](https://arxiv.org/abs/1603.09320).

FAISS has several ways for similarity search: L2 (Euclidean distance), cosine similarity. You can read more about their implementation on their [GitHub](https://github.com/facebookresearch/faiss/wiki/Getting-started#searching) page or [blog post](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/). They also published their own [best practice guide here](https://github.com/facebookresearch/faiss/wiki/Guidelines-to-choose-an-index).

If you'd like to read up more on the comparisons between vector libraries and databases, [here is a good blog post](https://weaviate.io/blog/vector-library-vs-vector-database#feature-comparison---library-versus-database).


**The overall workflow of FAISS is captured in the diagram below.**

<img width="100%" src="https://miro.medium.com/v2/resize:fit:1400/0*ouf0eyQskPeGWIGm">

Source: [How to use FAISS to build your first similarity search by Asna Shafiq](https://medium.com/loopio-tech/how-to-use-faiss-to-build-your-first-similarity-search-bf0f708aa772).


In [17]:
# O sentence_transformers é um framework Python desenvolvido para facilitar a geração 
# de embeddings de sentenças de alta qualidade, especialmente para tarefas de 
# NLP (Processamento de Linguagem Natural).

from sentence_transformers import InputExample

pdf_subset = pdf.head(1000)

def example_create_fn(doc1: pd.Series) -> InputExample:
    """
        Helper function that outputs a sentence_transformer guid, label and text
    """
    return InputExample(texts=[doc1])

In [18]:
faiss_train_examples = pdf_subset.apply(lambda x: example_create_fn(x['title']), axis=1).tolist()
print(faiss_train_examples)

[<sentence_transformers.readers.InputExample.InputExample object at 0x7f92fcd83e50>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f92fcd836a0>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f92fcd83c40>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f92fcd833a0>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f92fcd83970>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f92fcd83b80>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f92fcd83d60>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f92fcd83dc0>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f92fcd83f40>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f92fcd83a30>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f92fcd83be0>, <sentence_transformers.readers.InputExample.InputExample object at 0x7f92fc

In [19]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(
    'all-MiniLM-L6-v2',
    cache_folder=cache_dir
)

faiss_title_embedding = model.encode(pdf_subset.title.values.tolist())
print(faiss_title_embedding)

[[-0.11270548  0.04076539  0.02181413 ... -0.01874599 -0.03136874
   0.0682483 ]
 [-0.0218716  -0.03349996  0.07321802 ...  0.0336232  -0.00563894
  -0.00630979]
 [ 0.0160838   0.00279444 -0.01504422 ... -0.00706241  0.00905898
  -0.02835049]
 ...
 [ 0.01506929  0.04583015 -0.06114503 ... -0.07814188 -0.08025025
   0.01337818]
 [-0.07082234  0.0064382   0.0080932  ... -0.05520816 -0.03652045
   0.07594127]
 [-0.06321976  0.04461517 -0.07385812 ...  0.0655942   0.03276762
   0.09070998]]


In [20]:
len(faiss_title_embedding), len(faiss_title_embedding[0])

(1000, 384)

### Step 3: Saving embedding vectors to FAISS index
Below, we create the FAISS index object based on our embedding vectors, normalize vectors, and add these vectors to the FAISS index. 


In [21]:
import numpy as np
import faiss

pdf_to_index = pdf_subset.set_index(['id'], drop=False)
id_index = np.array(pdf_to_index.id.values).flatten().astype('int')
print(id_index)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

In [22]:
content_encoded_normalized = faiss_title_embedding.copy()
faiss.normalize_L2(content_encoded_normalized)

In [23]:
# Index1DMap translates search results to IDs: https://faiss.ai/cpp_api/file/IndexIDMap_8h.html#_CPPv4I0EN5faiss18IndexIDMapTemplateE
# The IndexFlatIP below builds index

index_content = faiss.IndexIDMap(faiss.IndexFlatIP(len(faiss_title_embedding[0])))
index_content.add_with_ids(content_encoded_normalized, id_index)

### Step 4: Search for relevant documents
We define a search function below to first vectorize our query text, and then search for the vectors with the closest distance. 

In [24]:
def search_content(query, pdf_to_index, k=5):
    query_vector = model.encode([query])
    faiss.normalize_L2(query_vector)

    top_k = index_content.search(query_vector, k)
    ids = top_k[1][0].tolist()
    similarities = top_k[0][0].tolist()
    results = pdf_to_index.loc[ids]
    results['similarities'] = similarities

    return results

display(search_content('bad', pdf_to_index))

,topic,link,domain,published_date,title,lang,id,similarities
id,,,,,,,,
322,TECHNOLOGY,https://www.forbes.com/sites/paultassi/2020/08...,forbes.com,2020-08-12 11:58:00,It’s Hard To Overstate How Damaging The ‘Halo ...,en,322,0.230659
474,TECHNOLOGY,https://www.forbes.com/sites/paultassi/2020/08...,forbes.com,2020-08-07 12:57:00,"A Warning About PS4’s ‘Avengers’ Beta, Live Th...",en,474,0.216699
608,HEALTH,https://bgr.com/posts/5862135/1fdeb6-a-bizarre...,bgr.com,2020-08-06 13:57:25,There's a weird new coronavirus symptom you mi...,en,608,0.208922
310,TECHNOLOGY,https://futurefive.co.nz/story/game-review-fal...,futurefive.co.nz,2020-08-11 20:56:38,Game review – Fall Guys: Ultimate Knockout,en,310,0.199409
763,HEALTH,https://www.canberratimes.com.au/story/6881029...,canberratimes.com.au,2020-08-17 01:57:00,Australia hits grim record with 25 deaths,en,763,0.198581
